In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

In [15]:
df_train = pd.read_csv('/content/Corona_NLP_train.csv',encoding="latin-1")
df_test = pd.read_csv('/content/Corona_NLP_test.csv')

In [16]:
df_test = df_test.drop(['UserName','ScreenName','Location','TweetAt'],axis=1)
df_train = df_train.drop(['UserName','ScreenName','Location','TweetAt'],axis=1)

In [17]:
max_features = 10000
max_len = 200

In [18]:
# 1. Fit tokenizer on training data
tokenizer = Tokenizer(num_words=max_features, oov_token="<OOV>")
tokenizer.fit_on_texts(df_train["OriginalTweet"])

# 2. Convert text to sequences of integers
x_train = tokenizer.texts_to_sequences(df_train["OriginalTweet"])
x_test = tokenizer.texts_to_sequences(df_test["OriginalTweet"])

# 3. Pad sequences to fixed length
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [19]:
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

In [20]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [21]:
# 1. Initialize and fit LabelEncoder
le = LabelEncoder()
le.fit(df_train["Sentiment"])   # learn classes from training data

num_classes = len(le.classes_)   # number of unique sentiment classes

# 2. Encode labels
y_train = tf.keras.utils.to_categorical(le.transform(df_train["Sentiment"]), num_classes)
y_test = tf.keras.utils.to_categorical(le.transform(df_test["Sentiment"]), num_classes)

# 3. Build LSTM model for multi-class classification
model = Sequential()
model.add(Embedding(input_dim=max_features, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation="softmax"))   # multi-class

# 4. Compile model
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [22]:
history = model.fit(
    x_train, y_train,
    batch_size=64,
    epochs=3,
    validation_data=(x_test, y_test)
)


Epoch 1/3
644/644 ━━━━━━━━━━━━━━━━━━━━ 427s 649ms/step - accuracy: 0.3943 - loss: 1.3722 - val_accuracy: 0.6796 - val_loss: 0.8648
Epoch 2/3
644/644 ━━━━━━━━━━━━━━━━━━━━ 439s 651ms/step - accuracy: 0.7484 - loss: 0.7128 - val_accuracy: 0.7533 - val_loss: 0.7132
Epoch 3/3
644/644 ━━━━━━━━━━━━━━━━━━━━ 418s 650ms/step - accuracy: 0.8183 - loss: 0.5425 - val_accuracy: 0.7712 - val_loss: 0.6728
